In [9]:
import numpy as np
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
import time
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from IPython.display import display
import time
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib


import gc

In [10]:
%load_ext Cython


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [11]:
pd.set_option("display.max_columns", 1000)


In [12]:
log = pd.read_csv("~/environment/log.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")
users = pd.read_csv("~/environment/users.tsv", sep="\t")


In [13]:

print(log.shape)
log.head()

(1452089, 7)


,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN


In [14]:
log.loc[:,"user_id_str"] = log["user_id"].apply(lambda x:str(x))
log.loc[:,"event_id_str"] = log["event_id"].apply(lambda x:str(x))
log.loc[:,"log_date"] = pd.to_datetime(log["time_stamp"])
log.loc[:,"log_year"] = log.loc[:,"log_date"].apply(lambda x:str(x.year))
log.loc[:,"log_month"] = log.loc[:,"log_date"].apply(lambda x:str(x.month))
log.loc[:,"log_day"] = log.loc[:,"log_date"].apply(lambda x:str(x.day))
log.loc[:,"log_weekday"] = log.loc[:,"log_date"].apply(lambda x:str(x.dayofweek))
log.loc[:,"log_hour"] = log.loc[:,"log_date"].apply(lambda x:str(x.hour))
log.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,log_date,log_year,log_month,log_day,log_weekday,log_hour
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,2015-03-19 09:15:50,2015,3,19,3,9
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,2017-06-05 12:33:17,2017,6,5,0,12
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,2017-06-05 12:37:09,2017,6,5,0,12
3,1,125296,2017-06-06 14:01:27,1,NaN,NaN,NaN,1,125296,2017-06-06 14:01:27,2017,6,6,1,14
4,1,125128,2017-06-06 14:03:57,1,NaN,NaN,NaN,1,125128,2017-06-06 14:03:57,2017,6,6,1,14


In [15]:
#action_typeが記録され始めた時以降
log = log.loc[log["log_date"] > pd.to_datetime("16/3/2017"),:]

In [16]:
data1 = pd.merge(log,events, on="event_id")
data = pd.merge(data1, users, on="user_id")
data = data.rename(columns = {"prefecture_x": "event_prefecture", "prefecture_y":"user_prefecture"})
data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,log_date,log_year,log_month,log_day,log_weekday,log_hour,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on
0,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,2017-06-05 12:33:17,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
1,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,2017-06-05 12:37:09,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
2,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,2017-06-05 12:32:04,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
3,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,2017-06-06 09:37:45,2017,6,6,1,9,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26
4,1,127600,2017-06-05 12:31:40,1,NaN,NaN,NaN,1,127600,2017-06-05 12:31:40,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26


In [17]:
data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_year"] = data.loc[:,"event_date"].apply(lambda x:str(x.year))
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,log_date,log_year,log_month,log_day,log_weekday,log_hour,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour
0,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,2017-06-05 12:33:17,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
1,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,2017-06-05 12:37:09,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
2,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,2017-06-05 12:32:04,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
3,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,2017-06-06 09:37:45,2017,6,6,1,9,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10
4,1,127600,2017-06-05 12:31:40,1,NaN,NaN,NaN,1,127600,2017-06-05 12:31:40,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10


In [18]:
data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,log_date,log_year,log_month,log_day,log_weekday,log_hour,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefecture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour,user_date,user_year,user_month
0,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,2017-06-05 12:33:17,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
1,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,2017-06-05 12:37:09,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
2,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,2017-06-05 12:32:04,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
3,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,2017-06-06 09:37:45,2017,6,6,1,9,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
4,1,127600,2017-06-05 12:31:40,1,NaN,NaN,NaN,1,127600,2017-06-05 12:31:40,2017,6,5,0,12,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6


In [19]:
features = ["female_age_lower","female_age_upper","male_age_lower","male_age_upper","event_prefecture","female_price","male_price","age", "gender", "user_prefecture","event_year", "event_month",
           "event_day","event_weekday", "event_hour", "user_year", "user_month"]
target = ["action_type"]

categorical = ["event_prefecture", "user_gender", "user_prefecture", 'event_year',"event_month","event_day","event_weekday", "event_hour", "user_year", "user_month"]



In [20]:
data = data[features + target]
#naの行消す
data = data.dropna()
data = data.reset_index(drop=True)
print(data.shape)
data.head()

(1144378, 18)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month,action_type
0,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6,1


In [21]:
X = data[features]
print(X.shape)
X.head()

(1144378, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6


In [22]:
def prefecture_labeling(x):
     #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["北海道","青森県","岩手県","宮城県","秋田県", 
"山形県", "福島県", "茨城県","栃木県","群馬県", 
"埼玉県","千葉県","東京都","神奈川県","新潟県", 
"富山県","石川県","福井県","山梨県","長野県", 
"岐阜県","静岡県","愛知県","三重県","滋賀県", 
"京都府","大阪府","兵庫県","奈良県","和歌山県", 
"鳥取県","島根県","岡山県","広島県","山口県", 
"徳島県","香川県","愛媛県","高知県","福岡県", 
"佐賀県","長崎県","熊本県","大分県","宮崎県", 
"鹿児島県","沖縄県",'その他（海外等）']
    label_encoder.fit(labels)
    le =label_encoder.transform(x)
    
    df = pd.DataFrame(le)
    column_name = ["prefecture"]
    df.columns = column_name
    
    return df
    

def gender_labeling(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["女性","男性"]
    label_encoder.fit(labels)
    le = label_encoder.transform(x)
    
    df = pd.DataFrame(le)
    column_name = ["user_gender"]
    df.columns = column_name
    
    return df
    
    

In [23]:
e_pre = prefecture_labeling(X["event_prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

(1144378, 1)


,event_prefecture
0,8
1,8
2,8
3,8
4,8


In [24]:
u_pre = prefecture_labeling(X["user_prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
print(u_pre.shape)
u_pre.head()

(1144378, 1)


,user_prefecture
0,43
1,43
2,43
3,43
4,43


In [25]:
gen = gender_labeling(X["gender"])
print(gen.shape)
gen.head()

(1144378, 1)


,user_gender
0,0
1,0
2,0
3,0
4,0


In [26]:
temp = X[list(set(features) - set(["user_prefecture", "event_prefecture", "gender"]))]
print(temp.shape)
temp = pd.concat((temp,gen), axis = 1)
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,u_pre), axis = 1)

X = temp
print(X.shape)
X.head()


(1144378, 14)
(1144378, 17)


,male_price,event_weekday,event_year,user_year,event_month,event_day,female_age_upper,male_age_lower,female_age_lower,female_price,event_hour,user_month,age,male_age_upper,user_gender,event_prefecture,user_prefecture
0,6000.0,6,2017,2012,6,11,36.0,26,24,2000.0,10,6,34,38.0,0,8,43
1,6000.0,6,2017,2012,6,11,36.0,26,24,2000.0,10,6,34,38.0,0,8,43
2,6000.0,6,2017,2012,6,11,36.0,26,24,2000.0,10,6,34,38.0,0,8,43
3,6000.0,6,2017,2012,6,11,36.0,26,24,2000.0,10,6,34,38.0,0,8,43
4,6000.0,6,2017,2012,6,11,36.0,26,24,2000.0,10,6,34,38.0,0,8,43


In [27]:
X = X.astype(np.int64)

In [28]:
y = data.loc[:,target]
print(y.shape)
y.head()

(1144378, 1)


,action_type
0,1
1,1
2,1
3,1
4,1


# 学習

In [29]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [30]:
y_train = np.array(y_train).reshape(-1)
y_train = y_train.astype(np.float64)
print(y_train.shape)
print(y_train.dtype)

(915502,)
float64


In [31]:
y_valid = np.array(y_valid).reshape(-1)
y_valid = y_valid.astype(np.float64)
print(y_valid.shape)
print(y_valid.dtype)

(228876,)
float64


lightGBMのAPI

In [32]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature = categorical)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature = categorical)

In [33]:
grid_params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [34]:
grid_gbm = lgb.cv(params, lgb_train,
                  num_boost_round=50,
            early_stopping_rounds=10)

NameError: name 'params' is not defined

In [ ]:
grid_gbm

In [35]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [36]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[1]	valid_0's l2: 0.423397
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.419733
[3]	valid_0's l2: 0.416681
[4]	valid_0's l2: 0.414161
[5]	valid_0's l2: 0.411996
[6]	valid_0's l2: 0.410238
[7]	valid_0's l2: 0.408724
[8]	valid_0's l2: 0.407491
[9]	valid_0's l2: 0.406321
[10]	valid_0's l2: 0.405333
[11]	valid_0's l2: 0.404449
[12]	valid_0's l2: 0.40364
[13]	valid_0's l2: 0.402892
[14]	valid_0's l2: 0.402207
[15]	valid_0's l2: 0.401669
[16]	valid_0's l2: 0.401129
[17]	valid_0's l2: 0.40054
[18]	valid_0's l2: 0.400079
[19]	valid_0's l2: 0.399705
[20]	valid_0's l2: 0.399384
[21]	valid_0's l2: 0.398907
[22]	valid_0's l2: 0.398604
[23]	valid_0's l2: 0.398254
[24]	valid_0's l2: 0.398
[25]	valid_0's l2: 0.397682
[26]	valid_0's l2: 0.397463
[27]	valid_0's l2: 0.397163
[28]	valid_0's l2: 0.396841
[29]	valid_0's l2: 0.39664
[30]	valid_0's l2: 0.396463
[31]	valid_0's l2: 0.396251
[32]	valid_0's l2: 0.396039
[33]	valid_0's l2: 0.395825
[34]	valid_0's l2: 0.395669
[

[288]	valid_0's l2: 0.377193
[289]	valid_0's l2: 0.377176
[290]	valid_0's l2: 0.377143
[291]	valid_0's l2: 0.377094
[292]	valid_0's l2: 0.377058
[293]	valid_0's l2: 0.377042
[294]	valid_0's l2: 0.377032
[295]	valid_0's l2: 0.376983
[296]	valid_0's l2: 0.376896
[297]	valid_0's l2: 0.376841
[298]	valid_0's l2: 0.376778
[299]	valid_0's l2: 0.376739
[300]	valid_0's l2: 0.376736
[301]	valid_0's l2: 0.376714
[302]	valid_0's l2: 0.376582
[303]	valid_0's l2: 0.376533
[304]	valid_0's l2: 0.376481
[305]	valid_0's l2: 0.376476
[306]	valid_0's l2: 0.376454
[307]	valid_0's l2: 0.376434
[308]	valid_0's l2: 0.376384
[309]	valid_0's l2: 0.376342
[310]	valid_0's l2: 0.37628
[311]	valid_0's l2: 0.376233
[312]	valid_0's l2: 0.37623
[313]	valid_0's l2: 0.376207
[314]	valid_0's l2: 0.376203
[315]	valid_0's l2: 0.376197
[316]	valid_0's l2: 0.376136
[317]	valid_0's l2: 0.376126
[318]	valid_0's l2: 0.376089
[319]	valid_0's l2: 0.376077
[320]	valid_0's l2: 0.376043
[321]	valid_0's l2: 0.376017
[322]	valid_0's 

[573]	valid_0's l2: 0.367445
[574]	valid_0's l2: 0.367435
[575]	valid_0's l2: 0.367434
[576]	valid_0's l2: 0.367414
[577]	valid_0's l2: 0.367404
[578]	valid_0's l2: 0.36737
[579]	valid_0's l2: 0.367335
[580]	valid_0's l2: 0.367325
[581]	valid_0's l2: 0.367311
[582]	valid_0's l2: 0.367303
[583]	valid_0's l2: 0.36727
[584]	valid_0's l2: 0.367261
[585]	valid_0's l2: 0.367258
[586]	valid_0's l2: 0.367246
[587]	valid_0's l2: 0.367245
[588]	valid_0's l2: 0.367168
[589]	valid_0's l2: 0.36716
[590]	valid_0's l2: 0.367142
[591]	valid_0's l2: 0.367101
[592]	valid_0's l2: 0.367098
[593]	valid_0's l2: 0.367051
[594]	valid_0's l2: 0.367001
[595]	valid_0's l2: 0.366947
[596]	valid_0's l2: 0.366928
[597]	valid_0's l2: 0.366879
[598]	valid_0's l2: 0.366844
[599]	valid_0's l2: 0.366749
[600]	valid_0's l2: 0.36673
[601]	valid_0's l2: 0.366708
[602]	valid_0's l2: 0.366702
[603]	valid_0's l2: 0.366672
[604]	valid_0's l2: 0.366636
[605]	valid_0's l2: 0.366588
[606]	valid_0's l2: 0.366548
[607]	valid_0's l2

[860]	valid_0's l2: 0.361795
[861]	valid_0's l2: 0.361793
[862]	valid_0's l2: 0.361751
[863]	valid_0's l2: 0.361753
[864]	valid_0's l2: 0.361749
[865]	valid_0's l2: 0.361747
[866]	valid_0's l2: 0.36172
[867]	valid_0's l2: 0.361701
[868]	valid_0's l2: 0.361662
[869]	valid_0's l2: 0.361637
[870]	valid_0's l2: 0.361622
[871]	valid_0's l2: 0.361608
[872]	valid_0's l2: 0.361595
[873]	valid_0's l2: 0.36158
[874]	valid_0's l2: 0.361569
[875]	valid_0's l2: 0.361558
[876]	valid_0's l2: 0.361542
[877]	valid_0's l2: 0.36153
[878]	valid_0's l2: 0.361522
[879]	valid_0's l2: 0.361503
[880]	valid_0's l2: 0.361495
[881]	valid_0's l2: 0.361485
[882]	valid_0's l2: 0.361486
[883]	valid_0's l2: 0.361482
[884]	valid_0's l2: 0.361445
[885]	valid_0's l2: 0.361434
[886]	valid_0's l2: 0.361428
[887]	valid_0's l2: 0.36142
[888]	valid_0's l2: 0.361412
[889]	valid_0's l2: 0.361403
[890]	valid_0's l2: 0.361401
[891]	valid_0's l2: 0.361391
[892]	valid_0's l2: 0.361363
[893]	valid_0's l2: 0.361354
[894]	valid_0's l2

[1139]	valid_0's l2: 0.358417
[1140]	valid_0's l2: 0.358393
[1141]	valid_0's l2: 0.358394
[1142]	valid_0's l2: 0.358389
[1143]	valid_0's l2: 0.358368
[1144]	valid_0's l2: 0.358354
[1145]	valid_0's l2: 0.358335
[1146]	valid_0's l2: 0.358322
[1147]	valid_0's l2: 0.358304
[1148]	valid_0's l2: 0.358289
[1149]	valid_0's l2: 0.358273
[1150]	valid_0's l2: 0.358259
[1151]	valid_0's l2: 0.358246
[1152]	valid_0's l2: 0.358227
[1153]	valid_0's l2: 0.35822
[1154]	valid_0's l2: 0.358214
[1155]	valid_0's l2: 0.358181
[1156]	valid_0's l2: 0.35817
[1157]	valid_0's l2: 0.358164
[1158]	valid_0's l2: 0.358151
[1159]	valid_0's l2: 0.358136
[1160]	valid_0's l2: 0.358117
[1161]	valid_0's l2: 0.358092
[1162]	valid_0's l2: 0.358078
[1163]	valid_0's l2: 0.358053
[1164]	valid_0's l2: 0.358018
[1165]	valid_0's l2: 0.357995
[1166]	valid_0's l2: 0.35799
[1167]	valid_0's l2: 0.357943
[1168]	valid_0's l2: 0.357932
[1169]	valid_0's l2: 0.357907
[1170]	valid_0's l2: 0.35788
[1171]	valid_0's l2: 0.357841
[1172]	valid_0

[1417]	valid_0's l2: 0.354588
[1418]	valid_0's l2: 0.35458
[1419]	valid_0's l2: 0.354579
[1420]	valid_0's l2: 0.354582
[1421]	valid_0's l2: 0.354581
[1422]	valid_0's l2: 0.354581
[1423]	valid_0's l2: 0.354578
[1424]	valid_0's l2: 0.354568
[1425]	valid_0's l2: 0.354558
[1426]	valid_0's l2: 0.354536
[1427]	valid_0's l2: 0.354528
[1428]	valid_0's l2: 0.354515
[1429]	valid_0's l2: 0.354496
[1430]	valid_0's l2: 0.354477
[1431]	valid_0's l2: 0.354462
[1432]	valid_0's l2: 0.354459
[1433]	valid_0's l2: 0.354411
[1434]	valid_0's l2: 0.354412
[1435]	valid_0's l2: 0.354383
[1436]	valid_0's l2: 0.354378
[1437]	valid_0's l2: 0.354366
[1438]	valid_0's l2: 0.354369
[1439]	valid_0's l2: 0.35437
[1440]	valid_0's l2: 0.35437
[1441]	valid_0's l2: 0.354367
[1442]	valid_0's l2: 0.354342
[1443]	valid_0's l2: 0.354342
[1444]	valid_0's l2: 0.354314
[1445]	valid_0's l2: 0.354278
[1446]	valid_0's l2: 0.354269
[1447]	valid_0's l2: 0.354262
[1448]	valid_0's l2: 0.354232
[1449]	valid_0's l2: 0.354211
[1450]	valid_

[1691]	valid_0's l2: 0.351854
[1692]	valid_0's l2: 0.351842
[1693]	valid_0's l2: 0.351839
[1694]	valid_0's l2: 0.351822
[1695]	valid_0's l2: 0.351808
[1696]	valid_0's l2: 0.351809
[1697]	valid_0's l2: 0.351784
[1698]	valid_0's l2: 0.351776
[1699]	valid_0's l2: 0.351771
[1700]	valid_0's l2: 0.351767
[1701]	valid_0's l2: 0.351759
[1702]	valid_0's l2: 0.351744
[1703]	valid_0's l2: 0.351746
[1704]	valid_0's l2: 0.351745
[1705]	valid_0's l2: 0.351682
[1706]	valid_0's l2: 0.351682
[1707]	valid_0's l2: 0.351668
[1708]	valid_0's l2: 0.35165
[1709]	valid_0's l2: 0.351646
[1710]	valid_0's l2: 0.35163
[1711]	valid_0's l2: 0.351631
[1712]	valid_0's l2: 0.351627
[1713]	valid_0's l2: 0.351624
[1714]	valid_0's l2: 0.351626
[1715]	valid_0's l2: 0.351616
[1716]	valid_0's l2: 0.351598
[1717]	valid_0's l2: 0.351596
[1718]	valid_0's l2: 0.351598
[1719]	valid_0's l2: 0.351593
[1720]	valid_0's l2: 0.351558
[1721]	valid_0's l2: 0.351549
[1722]	valid_0's l2: 0.351528
[1723]	valid_0's l2: 0.351518
[1724]	valid

[1966]	valid_0's l2: 0.349741
[1967]	valid_0's l2: 0.349736
[1968]	valid_0's l2: 0.349735
[1969]	valid_0's l2: 0.349717
[1970]	valid_0's l2: 0.349716
[1971]	valid_0's l2: 0.349711
[1972]	valid_0's l2: 0.349712
[1973]	valid_0's l2: 0.34971
[1974]	valid_0's l2: 0.349706
[1975]	valid_0's l2: 0.349706
[1976]	valid_0's l2: 0.349708
[1977]	valid_0's l2: 0.349687
[1978]	valid_0's l2: 0.349666
[1979]	valid_0's l2: 0.349655
[1980]	valid_0's l2: 0.349651
[1981]	valid_0's l2: 0.349651
[1982]	valid_0's l2: 0.349651
[1983]	valid_0's l2: 0.349649
[1984]	valid_0's l2: 0.349649
[1985]	valid_0's l2: 0.349635
[1986]	valid_0's l2: 0.349629
[1987]	valid_0's l2: 0.349627
[1988]	valid_0's l2: 0.349627
[1989]	valid_0's l2: 0.349625
[1990]	valid_0's l2: 0.349625
[1991]	valid_0's l2: 0.349619
[1992]	valid_0's l2: 0.349612
[1993]	valid_0's l2: 0.349613
[1994]	valid_0's l2: 0.349605
[1995]	valid_0's l2: 0.349603
[1996]	valid_0's l2: 0.349603
[1997]	valid_0's l2: 0.3496
[1998]	valid_0's l2: 0.349596
[1999]	valid_

[2241]	valid_0's l2: 0.347688
[2242]	valid_0's l2: 0.347674
[2243]	valid_0's l2: 0.347667
[2244]	valid_0's l2: 0.347651
[2245]	valid_0's l2: 0.347645
[2246]	valid_0's l2: 0.347628
[2247]	valid_0's l2: 0.347628
[2248]	valid_0's l2: 0.347624
[2249]	valid_0's l2: 0.347621
[2250]	valid_0's l2: 0.347587
[2251]	valid_0's l2: 0.347578
[2252]	valid_0's l2: 0.347577
[2253]	valid_0's l2: 0.347579
[2254]	valid_0's l2: 0.347579
[2255]	valid_0's l2: 0.347545
[2256]	valid_0's l2: 0.347521
[2257]	valid_0's l2: 0.347514
[2258]	valid_0's l2: 0.347511
[2259]	valid_0's l2: 0.347505
[2260]	valid_0's l2: 0.347503
[2261]	valid_0's l2: 0.347491
[2262]	valid_0's l2: 0.347477
[2263]	valid_0's l2: 0.347467
[2264]	valid_0's l2: 0.34746
[2265]	valid_0's l2: 0.347453
[2266]	valid_0's l2: 0.347449
[2267]	valid_0's l2: 0.347447
[2268]	valid_0's l2: 0.347447
[2269]	valid_0's l2: 0.347446
[2270]	valid_0's l2: 0.347442
[2271]	valid_0's l2: 0.347438
[2272]	valid_0's l2: 0.347435
[2273]	valid_0's l2: 0.347431
[2274]	vali

In [37]:
gbm.save_model('model.txt')


sklearnのAPI

In [ ]:
gbm = lgb.LGBMModel(objective ="regression")

In [ ]:
gbm.fit(X_train,y_train)

In [ ]:
gbm.best_score_

In [ ]:
gbm.best_iteration_

In [ ]:
pred = gbm.predict(X_valid)
mean_squared_error(pred, y_valid)

In [120]:
joblib.dump(gbm, 'gbm.pkl') 



['gbm.pkl']

rankを学習させる

In [121]:
gbm = lgb.LGBMModel(objective ="lambdarank")

In [122]:
gbm.fit(X_train,y_train)

LightGBMError: b'Lambdarank tasks require query information'

# predict

In [101]:
gbm = lgb.Booster(model_file='model.txt')



In [102]:
test = pd.read_csv("~/environment/test.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")
users = pd.read_csv("~/environment/users.tsv", sep="\t")


In [103]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


In [104]:

users.loc[:,"user_date"] = pd.to_datetime(users["created_on"])
users.loc[:,"user_year"] = users.loc[:,"user_date"].apply(lambda x:str(x.year))
users.loc[:,"user_month"] = users.loc[:,"user_date"].apply(lambda x:str(x.month))

print(users.shape)
users.head()

(69667, 8)


,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month
0,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6
1,2,31,男性,鳥取県,2012-07-12,2012-07-12,2012,7
2,3,32,男性,東京都,2012-07-15,2012-07-15,2012,7
3,4,30,男性,東京都,2012-07-17,2012-07-17,2012,7
4,5,29,女性,埼玉県,2012-07-17,2012-07-17,2012,7


In [105]:
events.head()

,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN


In [106]:
events.loc[:,"event_date"] = pd.to_datetime(events["event_start_at"])
events.loc[:,"event_year"] = events.loc[:,"event_date"].apply(lambda x:str(x.year))
events.loc[:,"event_month"] = events.loc[:,"event_date"].apply(lambda x:str(x.month))
events.loc[:,"event_day"] = events.loc[:,"event_date"].apply(lambda x:str(x.day))
events.loc[:,"event_weekday"] = events.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
events.loc[:,"event_hour"] = events.loc[:,"event_date"].apply(lambda x:str(x.hour))

print(events.shape)
events.head()

(158392, 17)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour
0,1,20,99.0,25,44.0,2015-03-08 18:00:00,宮城県,NaN,1500.0,5000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18
1,2,30,99.0,30,49.0,2015-03-01 15:00:00,富山県,NaN,1000.0,6500.0,NaN,2015-03-01 15:00:00,2015,3,1,6,15
2,3,22,99.0,24,39.0,2015-03-07 19:00:00,富山県,NaN,1000.0,6000.0,NaN,2015-03-07 19:00:00,2015,3,7,5,19
3,4,20,99.0,25,44.0,2015-03-08 18:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-08 18:00:00,2015,3,8,6,18
4,5,20,99.0,20,39.0,2015-03-14 19:00:00,新潟県,NaN,1000.0,6000.0,NaN,2015-03-14 19:00:00,2015,3,14,5,19


In [107]:
#action_type１が記録され始めた時以降
events = events.loc[events["event_date"] > pd.to_datetime("16/3/2017"),:]
events = events.reset_index(drop=True)
print(events.shape)
events.head()

(59791, 17)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour
0,59253,20,29.0,20,29.0,2017-03-18 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-03-18 18:30:00,2017,3,18,5,18
1,59266,25,39.0,25,39.0,2017-03-26 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-03-26 18:30:00,2017,3,26,6,18
2,59282,20,29.0,20,29.0,2017-04-02 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-02 18:30:00,2017,4,2,6,18
3,59340,25,39.0,25,39.0,2017-04-08 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-04-08 18:30:00,2017,4,8,5,18
4,59353,20,29.0,20,29.0,2017-04-15 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-15 18:30:00,2017,4,15,5,18


In [108]:
e_pre = prefecture_labeling(events["prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

(59791, 1)


,event_prefecture
0,43
1,43
2,43
3,43
4,43


In [109]:

events = pd.concat( [events,e_pre], axis = 1)
print(events.shape)
events.head()

(59791, 18)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour,event_prefecture
0,59253,20,29.0,20,29.0,2017-03-18 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-03-18 18:30:00,2017,3,18,5,18,43
1,59266,25,39.0,25,39.0,2017-03-26 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-03-26 18:30:00,2017,3,26,6,18,43
2,59282,20,29.0,20,29.0,2017-04-02 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-02 18:30:00,2017,4,2,6,18,43
3,59340,25,39.0,25,39.0,2017-04-08 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-04-08 18:30:00,2017,4,8,5,18,43
4,59353,20,29.0,20,29.0,2017-04-15 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-15 18:30:00,2017,4,15,5,18,43


In [110]:
u_pre = prefecture_labeling(users["prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
gen = gender_labeling(users["gender"])

users = pd.concat((users, u_pre), axis =1)
users = pd.concat((users, gen), axis = 1)
print(users.shape)
users.head()


(69667, 10)


,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month,user_prefecture,user_gender
0,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
1,2,31,男性,鳥取県,2012-07-12,2012-07-12,2012,7,46,1
2,3,32,男性,東京都,2012-07-15,2012-07-15,2012,7,27,1
3,4,30,男性,東京都,2012-07-17,2012-07-17,2012,7,27,1
4,5,29,女性,埼玉県,2012-07-17,2012-07-17,2012,7,8,0


In [111]:
user_event = events
user_event.loc[:,"user_id"] = 1

user_data = pd.merge(events, users, on = "user_id")

print(user_data.shape)
user_data.head()

(59791, 28)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour,event_prefecture,user_id,age,gender,prefecture_y,created_on,user_date,user_year,user_month,user_prefecture,user_gender
0,59253,20,29.0,20,29.0,2017-03-18 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-03-18 18:30:00,2017,3,18,5,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
1,59266,25,39.0,25,39.0,2017-03-26 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-03-26 18:30:00,2017,3,26,6,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
2,59282,20,29.0,20,29.0,2017-04-02 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-02 18:30:00,2017,4,2,6,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
3,59340,25,39.0,25,39.0,2017-04-08 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-04-08 18:30:00,2017,4,8,5,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
4,59353,20,29.0,20,29.0,2017-04-15 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-15 18:30:00,2017,4,15,5,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0


In [112]:
#userが登録された日以降のイベントだけ残す
user_data = user_data.loc[user_data["event_start_at"] > user_data["created_on"],:]
print(user_data.shape)
user_data.head()

(59791, 28)


,event_id,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,prefecture_x,first_published_at,female_price,male_price,interest,event_date,event_year,event_month,event_day,event_weekday,event_hour,event_prefecture,user_id,age,gender,prefecture_y,created_on,user_date,user_year,user_month,user_prefecture,user_gender
0,59253,20,29.0,20,29.0,2017-03-18 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-03-18 18:30:00,2017,3,18,5,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
1,59266,25,39.0,25,39.0,2017-03-26 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-03-26 18:30:00,2017,3,26,6,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
2,59282,20,29.0,20,29.0,2017-04-02 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-02 18:30:00,2017,4,2,6,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
3,59340,25,39.0,25,39.0,2017-04-08 18:30:00,静岡県,NaN,3000.0,7000.0,NaN,2017-04-08 18:30:00,2017,4,8,5,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0
4,59353,20,29.0,20,29.0,2017-04-15 18:30:00,静岡県,NaN,2500.0,8000.0,NaN,2017-04-15 18:30:00,2017,4,15,5,18,43,1,34,女性,静岡県,2012-06-26,2012-06-26,2012,6,43,0


In [113]:
features = ["female_age_lower","female_age_upper","male_age_lower","male_age_upper","event_prefecture","female_price","male_price","age", "user_gender", "user_prefecture","event_year", "event_month",
           "event_day","event_weekday", "event_hour", "user_year", "user_month"]
target = ["action_type"]

categorical = ["event_prefecture", "user_gender", "user_prefecture", 'event_year',"event_month","event_day","event_weekday", "event_hour", "user_year", "user_month"]


In [114]:


X = user_data[features]
X.loc[:,categorical] = X.loc[:,categorical].astype(np.int64)
print(X.shape)
X.head()

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(59791, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,user_gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,20,29.0,20,29.0,43,2500.0,8000.0,34,0,43,2017,3,18,5,18,2012,6
1,25,39.0,25,39.0,43,3000.0,7000.0,34,0,43,2017,3,26,6,18,2012,6
2,20,29.0,20,29.0,43,2500.0,8000.0,34,0,43,2017,4,2,6,18,2012,6
3,25,39.0,25,39.0,43,3000.0,7000.0,34,0,43,2017,4,8,5,18,2012,6
4,20,29.0,20,29.0,43,2500.0,8000.0,34,0,43,2017,4,15,5,18,2012,6


In [115]:
y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)


In [122]:
print(y_pred.shape)
y_pred


(59791,)


array([1.0093338 , 1.12301507, 1.03825247, ..., 0.94547807, 1.00629867,
       1.00629867])

In [117]:

user_list = test["user_id"]
event_list = events["event_id"]
print(len(user_list))
print(len(event_list))

2886
59791


In [118]:
###!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!user_id 69667以降はデモグラフィデータが存在しない!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
display(users.tail())
display(user_list.loc[user_list > 69667])
print(user_list.loc[user_list > 69667].shape)

##とりあえずデモグラフィデータが存在するユーザだけに対して予測する
user_list = user_list.loc[user_list < 69667]
display(user_list.tail())
print(user_list.shape)

,user_id,age,gender,prefecture,created_on,user_date,user_year,user_month,user_prefecture,user_gender
69662,69663,24,女性,埼玉県,2017-09-23,2017-09-23,2017,9,8,0
69663,69664,23,女性,宮城県,2017-09-23,2017-09-23,2017,9,12,0
69664,69665,32,男性,岡山県,2017-09-23,2017-09-23,2017,9,19,1
69665,69666,23,女性,東京都,2017-09-23,2017-09-23,2017,9,27,0
69666,69667,42,男性,千葉県,2017-09-23,2017-09-23,2017,9,6,1


2556    69668
2557    69671
2558    69672
2559    69673
2560    69674
2561    69675
2562    69676
2563    69678
2564    69679
2565    69680
2566    69681
2567    69682
2568    69683
2569    69684
2570    69685
2571    69686
2572    69687
2573    69688
2574    69689
2575    69690
2576    69691
2577    69692
2578    69693
2579    69694
2580    69695
2581    69696
2582    69697
2583    69698
2584    69699
2585    69700
        ...  
2856    69971
2857    69972
2858    69973
2859    69974
2860    69975
2861    69976
2862    69977
2863    69978
2864    69979
2865    69980
2866    69981
2867    69982
2868    69983
2869    69984
2870    69985
2871    69986
2872    69987
2873    69988
2874    69989
2875    69990
2876    69991
2877    69992
2878    69993
2879    69994
2880    69995
2881    69996
2882    69997
2883    69998
2884    69999
2885    70000
Name: user_id, Length: 330, dtype: int64

(330,)


2551    69649
2552    69654
2553    69660
2554    69663
2555    69664
Name: user_id, dtype: int64

(2556,)


In [123]:
result = pd.DataFrame({"event_id":user_data["event_id"].values})
result.loc[:,"predict"] = y_pred
result = result.sort_values(by=["predict"], ascending=False)
result = result.reset_index(drop = True)
result.loc[:,"rank"] = result.index.values + 1

In [124]:
print(result.shape)
result.head()

(59791, 3)


,event_id,predict,rank
0,124772,2.072570,1
1,154718,2.030749,2
2,132702,2.030749,3
3,149106,2.030749,4
4,135061,2.025043,5


In [127]:
def predict_user(user_id):
    
    user_event.loc[:,"user_id"] = user_id

    user_data = pd.merge(events, users, on = "user_id")
    
    #userが登録された日以降のイベントだけ残す
    user_data = user_data.loc[user_data["event_start_at"] > user_data["created_on"],:]
    
    
    X = user_data[features]
    X.loc[:,categorical] = X.loc[:,categorical].astype(np.int64)
    
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    
    result = pd.DataFrame({"event_id":user_data["event_id"].values})
    
    print(user_data.shape)
    print(X.shape)
    print(y_pred.shape)
    print(result.shape)
    display(result.head())
    
    result.loc[:,"predict"] = y_pred
    result.loc[:,"user_id"] = user_id

    result = result.sort_values(by=["predict"], ascending=False)
    result = result.reset_index(drop = True)
    result.loc[:,"rank"] = result.index.values + 1
    
    return result


In [128]:
start = time.time() #時間計測

predict = predict_user(41635)

elapsed_time = time.time() - start #時間計測ここまで

print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(59706, 28)
(59706, 17)
(59706,)
(59706, 1)


,event_id
0,59253
1,59266
2,59282
3,59340
4,59353


elapsed_time:7.647027015686035[sec]


In [129]:
predict.head(20)

,event_id,predict,user_id,rank
0,156245,2.240328,41635,1
1,157840,2.240328,41635,2
2,157843,2.240328,41635,3
3,157844,2.240328,41635,4
4,158002,2.240328,41635,5
5,157979,2.240328,41635,6
6,157929,2.240328,41635,7
7,157932,2.240328,41635,8
8,156236,2.240328,41635,9
9,156414,2.230961,41635,10


In [96]:


k = 20
i = 0
recommends = []

for user in user_list:
    
    start = time.time() #時間計測

    
    i = i +1
    print("user_id:"+str(user))
    print("進捗:"+str(i*100/len(user_list))+"%") 
    
    result = predict_user(user)
    
    recommend_by_user = result.loc[:k-1,:]
    recommends.append(recommend_by_user)
    
    elapsed_time = time.time() - start #時間計測ここまで
    print ("予想残り時間")
    print(elapsed_time * (len(user_list)-i))


user_id:1
進捗:0.03912363067292645%


/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


予想残り時間
19124.666932821274
user_id:6
進捗:0.0782472613458529%
予想残り時間
19067.380523204803
user_id:23
進捗:0.11737089201877934%
予想残り時間
18679.486358642578
user_id:43
進捗:0.1564945226917058%
予想残り時間
18626.712202072144
user_id:46
進捗:0.19561815336463223%
予想残り時間
21947.731358528137
user_id:48
進捗:0.2347417840375587%
予想残り時間
18784.010553359985
user_id:76
進捗:0.27386541471048514%
予想残り時間
18999.371468305588
user_id:79
進捗:0.3129890453834116%
予想残り時間
19267.256216049194
user_id:104
進捗:0.352112676056338%
予想残り時間
19102.459314107895
user_id:139
進捗:0.39123630672926446%
予想残り時間
18883.70520734787
user_id:152
進捗:0.4303599374021909%
予想残り時間
18658.376383781433
user_id:198
進捗:0.4694835680751174%
予想残り時間
19258.860443115234
user_id:206
進捗:0.5086071987480438%
予想残り時間
18622.6514108181
user_id:233
進捗:0.5477308294209703%
予想残り時間
18688.09369277954
user_id:242
進捗:0.5868544600938967%
予想残り時間
18656.089265584946
user_id:247
進捗:0.6259780907668232%
予想残り時間
19020.086612701416
user_id:263
進捗:0.6651017214397497%
予想残り時間
19056.50465130806
user_id:

予想残り時間
18360.829782009125
user_id:2674
進捗:5.39906103286385%
予想残り時間
18086.21170949936
user_id:2711
進捗:5.438184663536776%
予想残り時間
18108.06916642189
user_id:2777
進捗:5.477308294209703%
予想残り時間
18529.316425323486
user_id:2803
進捗:5.516431924882629%
予想残り時間
17949.03408408165
user_id:2899
進捗:5.555555555555555%
予想残り時間
18028.63069677353
user_id:2902
進捗:5.594679186228482%
予想残り時間
18014.901298999786
user_id:2904
進捗:5.633802816901408%
予想残り時間
18187.839363098145
user_id:2908
進捗:5.672926447574335%
予想残り時間
18179.17731165886
user_id:2987
進捗:5.712050078247262%
予想残り時間
17924.30547475815
user_id:2997
進捗:5.751173708920188%
予想残り時間
18200.305884361267
user_id:3011
進捗:5.790297339593114%
予想残り時間
18195.524864196777
user_id:3031
進捗:5.829420970266041%
予想残り時間
18256.066710472107
user_id:3039
進捗:5.868544600938967%
予想残り時間
18217.29130268097
user_id:3050
進捗:5.907668231611893%
予想残り時間
18237.67622232437
user_id:3062
進捗:5.94679186228482%
予想残り時間
18495.994863510132
user_id:3063
進捗:5.985915492957746%
予想残り時間
18375.212946653366
user_id:

予想残り時間
17274.686247110367
user_id:5432
進捗:10.719874804381847%
予想残り時間
17164.404385089874
user_id:5434
進捗:10.758998435054773%
予想残り時間
17033.5137257576
user_id:5500
進捗:10.7981220657277%
予想残り時間
16997.073125839233
user_id:5509
進捗:10.837245696400625%
予想残り時間
16886.00192117691
user_id:5511
進捗:10.876369327073553%
予想残り時間
17013.282417297363
user_id:5546
進捗:10.915492957746478%
予想残り時間
16942.572284460068
user_id:5592
進捗:10.954616588419405%
予想残り時間
16766.731143951416
user_id:5614
進捗:10.993740219092333%
予想残り時間
17102.520608901978
user_id:5623
進捗:11.032863849765258%
予想残り時間
17233.77148103714
user_id:5650
進捗:11.071987480438185%
予想残り時間
16641.229885339737
user_id:5655
進捗:11.11111111111111%
予想残り時間
16749.971565246582
user_id:5660
進捗:11.150234741784038%
予想残り時間
17192.93825697899
user_id:5738
進捗:11.189358372456963%
予想残り時間
16692.83550262451
user_id:5743
進捗:11.22848200312989%
予想残り時間
16466.589510917664
user_id:5745
進捗:11.267605633802816%
予想残り時間
16740.091332435608
user_id:5753
進捗:11.306729264475743%
予想残り時間
16988.23742

...

予想残り時間
9976.907025814056
user_id:24127
進捗:47.06572769953052%
予想残り時間
10037.287524461746
user_id:24154
進捗:47.104851330203445%
予想残り時間
10286.5546875
user_id:24200
進捗:47.14397496087637%
予想残り時間
10253.09651684761
user_id:24238
進捗:47.183098591549296%
予想残り時間
10030.291521549225
user_id:24243
進捗:47.22222222222222%
予想残り時間
9993.35114312172
user_id:24315
進捗:47.26134585289515%
予想残り時間
10009.21232509613
user_id:24316
進捗:47.30046948356807%
予想残り時間
10162.427199840546
user_id:24379
進捗:47.339593114241005%
予想残り時間
9931.42664194107
user_id:24419
進捗:47.37871674491393%
予想残り時間
9939.887489080429
user_id:24434
進捗:47.417840375586856%
予想残り時間
10050.08967590332
user_id:24460
進捗:47.45696400625978%
予想残り時間
10010.935565710068
user_id:24508
進捗:47.49608763693271%
予想残り時間
9959.083431720734
user_id:24585
進捗:47.53521126760563%
予想残り時間
9831.935146093369
user_id:24626
進捗:47.57433489827856%
予想残り時間
10297.942094802856
user_id:24641
進捗:47.61345852895149%
予想残り時間
9930.233594179153
user_id:24710
進捗:47.652582159624416%
予想残り時間
9841.56263780

予想残り時間
9027.167242527008
user_id:27436
進捗:52.347417840375584%
予想残り時間
9254.146845817566
user_id:27441
進捗:52.38654147104851%
予想残り時間
9032.076299905777
user_id:27444
進捗:52.42566510172144%
予想残り時間
9149.562545776367
user_id:27470
進捗:52.46478873239437%
予想残り時間
9234.908316135406
user_id:27482
進捗:52.50391236306729%
予想残り時間
9186.036586284637
user_id:27508
進捗:52.54303599374022%
予想残り時間
9040.745316028595
user_id:27517
進捗:52.582159624413144%
予想残り時間
9175.757377624512
user_id:27552
進捗:52.62128325508607%
予想残り時間
9370.199637174606
user_id:27572
進捗:52.660406885758995%
予想残り時間
8990.961291790009
user_id:27573
進捗:52.69953051643193%
予想残り時間
9024.397114276886
user_id:27574
進捗:52.73865414710485%
予想残り時間
9150.266254425049
user_id:27576
進捗:52.77777777777778%
予想残り時間
9105.982857704163
user_id:27636
進捗:52.816901408450704%
予想残り時間
9138.27564239502
user_id:27705
進捗:52.85602503912363%
予想残り時間
9135.060002803802
user_id:27742
進捗:52.895148669796555%
予想残り時間
9328.733119010925
user_id:27756
進捗:52.93427230046948%
予想残り時間
9190.71103620

予想残り時間
8344.847494125366
user_id:32588
進捗:57.629107981220656%
予想残り時間
8076.639539480209
user_id:32690
進捗:57.66823161189358%
予想残り時間
8205.833946228027
user_id:32763
進捗:57.70735524256651%
予想残り時間
8263.595756292343
user_id:32790
進捗:57.74647887323944%
予想残り時間
8148.242425918579
user_id:33008
進捗:57.785602503912365%
予想残り時間
8215.15519452095
user_id:33016
進捗:57.82472613458529%
予想残り時間
8014.590678215027
user_id:33048
進捗:57.863849765258216%
予想残り時間
7975.758095026016
user_id:33053
進捗:57.90297339593114%
予想残り時間
8043.674799919128
user_id:33064
進捗:57.94209702660407%
予想残り時間
8041.939616203308
user_id:33109
進捗:57.98122065727699%
予想残り時間
7993.739321708679
user_id:33129
進捗:58.020344287949925%
予想残り時間
7953.949316740036
user_id:33180
進捗:58.05946791862285%
予想残り時間
8163.113624572754
user_id:33333
進捗:58.098591549295776%
予想残り時間
8046.385987043381
user_id:33339
進捗:58.1377151799687%
予想残り時間
8093.789329528809
user_id:33445
進捗:58.17683881064163%
予想残り時間
7900.211367845535
user_id:33453
進捗:58.21596244131455%
予想残り時間
8202.148149490

予想残り時間
7069.429707527161
user_id:40895
進捗:62.8716744913928%
予想残り時間
7079.463334321976
user_id:40916
進捗:62.91079812206573%
予想残り時間
7069.283932685852
user_id:40949
進捗:62.94992175273865%
予想残り時間
7142.732616424561
user_id:41019
進捗:62.98904538341158%
予想残り時間
7122.925952911377
user_id:41041
進捗:63.028169014084504%
予想残り時間
7072.471894025803
user_id:41097
進捗:63.06729264475744%
予想残り時間
7133.227478027344
user_id:41128
進捗:63.10641627543036%
予想残り時間
7029.110080480576
user_id:41269
進捗:63.14553990610329%
予想残り時間
7122.883433818817
user_id:41314
進捗:63.18466353677621%
予想残り時間
7245.848117113113
user_id:41318
進捗:63.22378716744914%
予想残り時間
7213.457827568054
user_id:41321
進捗:63.262910798122064%
予想残り時間
7148.027084827423
user_id:41332
進捗:63.30203442879499%
予想残り時間
7096.501414299011
user_id:41361
進捗:63.341158059467915%
予想残り時間
7145.114242076874
user_id:41362
進捗:63.38028169014085%
予想残り時間
7112.583194732666
user_id:41387
進捗:63.41940532081377%
予想残り時間
7174.1300320625305
user_id:41415
進捗:63.4585289514867%
予想残り時間
7124.2854681015

ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
f = open('recommend_lightGBM.txt', 'w')
for x in recommends:
    f.write(str(x) + "\n")
f.close()

In [ ]:
sub = pd.DataFrame()

for recommend in recommends:
    
    sub = pd.concat([sub,recommend])
    

In [ ]:
sub.head(50)

In [ ]:
sub.tail()

In [ ]:
sub = sub.loc[:,["user_id","event_id","rank"]]
print(sub.shape)
sub.head(50)

In [ ]:
sample_submit = pd.read_csv("~/environment/sample_submit.tsv", sep="\t", header = None)
sample_submit.columns = sub.columns
print(sample_submit.shape)
sample_submit.head()

In [ ]:
submmition = pd.concat([sub,sample_submit.loc[sample_submit["user_id"]> 69667,:]])
print(submmition.shape)
submmition.head()

In [ ]:
submmition.to_csv("sub_lightgbm.tsv",header=None,index=None,sep = "\t")

# 以下工事中

# categorycaなfeatureをダミー化

しなくてよかったっぽい
https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

In [46]:
#categoricalな特徴をダミー化する
#各特徴に専用の関数を作る

def month_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    cat = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"month"+str(x),cat))
    
    df.columns = column_name
    
    return df
    
def day_dummying(x):
    onehot_encoder = OneHotEncoder()
    
    cat = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"day"+str(x),cat))
    
    df.columns = column_name
    
    return df

def year_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [2012,2013,2014,2015,2016,2017]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"year"+str(x),cat))
    
    df.columns = column_name
    
    return df
    

def weekday_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [0,1,2,3,4,5,6]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"week_day"+str(x),cat))
    
    df.columns = column_name
    
    return df

def hour_dummying(x):
    
    onehot_encoder = OneHotEncoder()
    
    cat = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    
    category = np.array(cat).reshape(-1,1)
    
    onehot_encoder.fit(category)
    
    mat =  onehot_encoder.transform(np.array([x]).reshape(-1,1)).toarray()
    
    df = pd.DataFrame(mat)
    
    column_name = list(map(lambda x:"hour"+str(x),cat))
    
    df.columns = column_name
    
    return df
    
    
    

In [47]:
def gender_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["女性","男性"]
    label_encoder.fit(labels)
    le = label_encoder.transform(x)
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arange(len(labels)).reshape(-1,1))
    onehot_encoder.fit(category)
    
    mat = onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()
    df = pd.DataFrame(mat)
    column_name = label_encoder.classes_
    df.columns = column_name
    
    return df
    

def prefecture_dummying(x):
    
    #文字列をラベリング
    label_encoder = LabelEncoder()
    labels = ["北海道","青森県","岩手県","宮城県","秋田県", 
"山形県", "福島県", "茨城県","栃木県","群馬県", 
"埼玉県","千葉県","東京都","神奈川県","新潟県", 
"富山県","石川県","福井県","山梨県","長野県", 
"岐阜県","静岡県","愛知県","三重県","滋賀県", 
"京都府","大阪府","兵庫県","奈良県","和歌山県", 
"鳥取県","島根県","岡山県","広島県","山口県", 
"徳島県","香川県","愛媛県","高知県","福岡県", 
"佐賀県","長崎県","熊本県","大分県","宮崎県", 
"鹿児島県","沖縄県",'その他（海外等）']
    label_encoder.fit(labels)
    le =label_encoder.transform(x)
    
    label_encoder.classes_
    
    #数値をonehot vectorへ
    
    onehot_encoder = OneHotEncoder()
    category = np.array(np.arange(len(labels))).reshape(-1,1)
    onehot_encoder.fit(category)
    
    mat = onehot_encoder.transform(np.array([le]).reshape(-1,1)).toarray()
    df = pd.DataFrame(mat)
    column_name = label_encoder.classes_
    df.columns = column_name
    
    return df

categorycalな列を変換

In [48]:
e_pre = prefecture_dummying(X["event_prefercture"])
e_pre.columns = "e_" + e_pre.columns.values
print(e_pre.shape)
e_pre.head()

KeyError: 'event_prefercture'

In [49]:
u_pre = prefecture_dummying(X["user_prefecture"])
u_pre.columns = "u_" + u_pre.columns.values
print(u_pre.shape)
u_pre.head()

KeyError: 'user_prefecture'

In [50]:
gender = gender_dummying(X["gender"])
print(gender.shape)
gender.head()

KeyError: 'gender'

In [16]:
u_year = year_dummying(X["user_year"])
u_year.columns = "u_" + u_year.columns.values
print(u_year.shape)
u_year.head()


(1450405, 6)


,u_year2012,u_year2013,u_year2014,u_year2015,u_year2016,u_year2017
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
u_month = month_dummying(X["user_month"])
u_month.columns = "u_" + u_month.columns.values
print(u_month.shape)
u_month.head()

(1450405, 12)


,u_month1,u_month2,u_month3,u_month4,u_month5,u_month6,u_month7,u_month8,u_month9,u_month10,u_month11,u_month12
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
e_year = year_dummying(X["event_year"])
e_year.columns = "e_" + e_year.columns.values
print(e_year.shape)
e_year.head()


(1450405, 6)


,e_year2012,e_year2013,e_year2014,e_year2015,e_year2016,e_year2017
0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
e_month = month_dummying(X["event_month"])
e_month.columns = "e_" + e_month.columns.values
print(e_month.shape)
e_month.head()


(1450405, 12)


,e_month1,e_month2,e_month3,e_month4,e_month5,e_month6,e_month7,e_month8,e_month9,e_month10,e_month11,e_month12
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
e_day = day_dummying(X["event_day"])
e_day.columns = "e_" + e_day.columns.values
print(e_day.shape)
e_day.head()

(1450405, 31)


,e_day1,e_day2,e_day3,e_day4,e_day5,e_day6,e_day7,e_day8,e_day9,e_day10,e_day11,e_day12,e_day13,e_day14,e_day15,e_day16,e_day17,e_day18,e_day19,e_day20,e_day21,e_day22,e_day23,e_day24,e_day25,e_day26,e_day27,e_day28,e_day29,e_day30,e_day31
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
e_weekday = weekday_dummying(X["event_weekday"])
e_weekday.columns = "e_" + e_weekday.columns.values
print(e_weekday.shape)
e_weekday.head()

(1450405, 7)


,e_week_day0,e_week_day1,e_week_day2,e_week_day3,e_week_day4,e_week_day5,e_week_day6
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
e_hour = hour_dummying(X["event_hour"])
e_hour.columns = "e_" + e_hour.columns.values
print(e_hour.shape)
e_hour.head()

(1450405, 25)


,e_hour0,e_hour1,e_hour2,e_hour3,e_hour4,e_hour5,e_hour6,e_hour7,e_hour8,e_hour9,e_hour10,e_hour11,e_hour12,e_hour13,e_hour14,e_hour15,e_hour16,e_hour17,e_hour18,e_hour19,e_hour20,e_hour21,e_hour22,e_hour23,e_hour24
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
temp = X[list(set(features) - set(categorical))]
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,gender), axis  =1 )
temp = pd.concat((temp,u_pre), axis = 1)
temp = pd.concat((temp, e_year), axis = 1)
temp = pd.concat((temp, e_month), axis = 1)
temp = pd.concat((temp, e_day), axis = 1)
temp = pd.concat((temp, e_weekday), axis = 1)
temp = pd.concat((temp, e_hour), axis = 1)
temp = pd.concat((temp, u_year), axis = 1)
temp = pd.concat((temp, u_month), axis = 1)
print(temp.shape)
temp.head()

(1450405, 204)


,age,female_price,male_age_upper,female_age_upper,male_age_lower,female_age_lower,male_price,e_その他（海外等）,e_三重県,e_京都府,e_佐賀県,e_兵庫県,e_北海道,e_千葉県,e_和歌山県,e_埼玉県,e_大分県,e_大阪府,e_奈良県,e_宮城県,e_宮崎県,e_富山県,e_山口県,e_山形県,e_山梨県,e_岐阜県,e_岡山県,e_岩手県,e_島根県,e_広島県,e_徳島県,e_愛媛県,e_愛知県,e_新潟県,e_東京都,e_栃木県,e_沖縄県,e_滋賀県,e_熊本県,e_石川県,e_神奈川県,e_福井県,e_福岡県,e_福島県,e_秋田県,e_群馬県,e_茨城県,e_長崎県,e_長野県,e_青森県,e_静岡県,e_香川県,e_高知県,e_鳥取県,e_鹿児島県,女性,男性,u_その他（海外等）,u_三重県,u_京都府,u_佐賀県,u_兵庫県,u_北海道,u_千葉県,u_和歌山県,u_埼玉県,u_大分県,u_大阪府,u_奈良県,u_宮城県,u_宮崎県,u_富山県,u_山口県,u_山形県,u_山梨県,u_岐阜県,u_岡山県,u_岩手県,u_島根県,u_広島県,u_徳島県,u_愛媛県,u_愛知県,u_新潟県,u_東京都,u_栃木県,u_沖縄県,u_滋賀県,u_熊本県,u_石川県,u_神奈川県,u_福井県,u_福岡県,u_福島県,u_秋田県,u_群馬県,u_茨城県,u_長崎県,u_長野県,u_青森県,u_静岡県,u_香川県,u_高知県,u_鳥取県,u_鹿児島県,e_year2012,e_year2013,e_year2014,e_year2015,e_year2016,e_year2017,e_month1,e_month2,e_month3,e_month4,e_month5,e_month6,e_month7,e_month8,e_month9,e_month10,e_month11,e_month12,e_day1,e_day2,e_day3,e_day4,e_day5,e_day6,e_day7,e_day8,e_day9,e_day10,e_day11,e_day12,e_day13,e_day14,e_day15,e_day16,e_day17,e_day18,e_day19,e_day20,e_day21,e_day22,e_day23,e_day24,e_day25,e_day26,e_day27,e_day28,e_day29,e_day30,e_day31,e_week_day0,e_week_day1,e_week_day2,e_week_day3,e_week_day4,e_week_day5,e_week_day6,e_hour0,e_hour1,e_hour2,e_hour3,e_hour4,e_hour5,e_hour6,e_hour7,e_hour8,e_hour9,e_hour10,e_hour11,e_hour12,e_hour13,e_hour14,e_hour15,e_hour16,e_hour17,e_hour18,e_hour19,e_hour20,e_hour21,e_hour22,e_hour23,e_hour24,u_year2012,u_year2013,u_year2014,u_year2015,u_year2016,u_year2017,u_month1,u_month2,u_month3,u_month4,u_month5,u_month6,u_month7,u_month8,u_month9,u_month10,u_month11,u_month12
0,34,4000.0,39.0,39.0,27,27,6500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34,2000.0,38.0,36.0,26,24,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34,2000.0,38.0,36.0,26,24,6000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

# 学習

In [25]:
X = temp

In [26]:
y = data.loc[:,target]
print(y.shape)
y.head()

(1450405, 1)


,action_type
0,3
1,1
2,1
3,1
4,1


In [27]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [28]:
y_train = np.array(y_train).reshape(-1)
y_train = y_train.astype(np.float64)
print(y_train.shape)
print(y_train.dtype)

(1160324,)
float64


In [29]:
y_valid = np.array(y_valid).reshape(-1)
y_valid = y_valid.astype(np.float64)
print(y_valid.shape)
print(y_valid.dtype)

(290081,)
float64


In [48]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=['c3'])
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

In [55]:
lgb_train.get_field("gender")

Exception: Cannot get gender before construct Dataset

In [38]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mean_squared_error'},
        'learning_rate': 0.1,
        'num_leaves': 23,
        'min_data_in_leaf': 1,
        'num_iteration': 10000,
        'verbose': 0
}


In [39]:
gbm = lgb.train(params,
            lgb_train,
            num_boost_round=50,
            valid_sets=lgb_eval,
            early_stopping_rounds=10)


/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.552014
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.519943
[3]	valid_0's l2: 0.493896
[4]	valid_0's l2: 0.47265
[5]	valid_0's l2: 0.455379
[6]	valid_0's l2: 0.44132
[7]	valid_0's l2: 0.429646
[8]	valid_0's l2: 0.420306
[9]	valid_0's l2: 0.412629
[10]	valid_0's l2: 0.406207
[11]	valid_0's l2: 0.401102
[12]	valid_0's l2: 0.39677
[13]	valid_0's l2: 0.393205
[14]	valid_0's l2: 0.390245
[15]	valid_0's l2: 0.387918
[16]	valid_0's l2: 0.385837
[17]	valid_0's l2: 0.384236
[18]	valid_0's l2: 0.382449
[19]	valid_0's l2: 0.381205
[20]	valid_0's l2: 0.379905
[21]	valid_0's l2: 0.379007
[22]	valid_0's l2: 0.377982
[23]	valid_0's l2: 0.377311
[24]	valid_0's l2: 0.376618
[25]	valid_0's l2: 0.376054
[26]	valid_0's l2: 0.375384
[27]	valid_0's l2: 0.374938
[28]	valid_0's l2: 0.374442
[29]	valid_0's l2: 0.374046
[30]	valid_0's l2: 0.373613
[31]	valid_0's l2: 0.373235
[32]	valid_0's l2: 0.372919
[33]	valid_0's l2: 0.372652
[34]	valid_0's l2: 0.37239

[292]	valid_0's l2: 0.357452
[293]	valid_0's l2: 0.357413
[294]	valid_0's l2: 0.357363
[295]	valid_0's l2: 0.357342
[296]	valid_0's l2: 0.357293
[297]	valid_0's l2: 0.357288
[298]	valid_0's l2: 0.357281
[299]	valid_0's l2: 0.357277
[300]	valid_0's l2: 0.35726
[301]	valid_0's l2: 0.357242
[302]	valid_0's l2: 0.357225
[303]	valid_0's l2: 0.357157
[304]	valid_0's l2: 0.357143
[305]	valid_0's l2: 0.35712
[306]	valid_0's l2: 0.357103
[307]	valid_0's l2: 0.357089
[308]	valid_0's l2: 0.357066
[309]	valid_0's l2: 0.357062
[310]	valid_0's l2: 0.357054
[311]	valid_0's l2: 0.357045
[312]	valid_0's l2: 0.357041
[313]	valid_0's l2: 0.357033
[314]	valid_0's l2: 0.35699
[315]	valid_0's l2: 0.356975
[316]	valid_0's l2: 0.356938
[317]	valid_0's l2: 0.356929
[318]	valid_0's l2: 0.356916
[319]	valid_0's l2: 0.356906
[320]	valid_0's l2: 0.356866
[321]	valid_0's l2: 0.356829
[322]	valid_0's l2: 0.356827
[323]	valid_0's l2: 0.356798
[324]	valid_0's l2: 0.356779
[325]	valid_0's l2: 0.356751
[326]	valid_0's l

[576]	valid_0's l2: 0.351405
[577]	valid_0's l2: 0.351368
[578]	valid_0's l2: 0.351359
[579]	valid_0's l2: 0.351353
[580]	valid_0's l2: 0.351336
[581]	valid_0's l2: 0.351325
[582]	valid_0's l2: 0.351304
[583]	valid_0's l2: 0.3513
[584]	valid_0's l2: 0.351295
[585]	valid_0's l2: 0.351286
[586]	valid_0's l2: 0.351286
[587]	valid_0's l2: 0.351284
[588]	valid_0's l2: 0.351278
[589]	valid_0's l2: 0.351271
[590]	valid_0's l2: 0.351267
[591]	valid_0's l2: 0.351224
[592]	valid_0's l2: 0.351215
[593]	valid_0's l2: 0.351204
[594]	valid_0's l2: 0.351199
[595]	valid_0's l2: 0.351192
[596]	valid_0's l2: 0.351185
[597]	valid_0's l2: 0.351174
[598]	valid_0's l2: 0.35116
[599]	valid_0's l2: 0.351155
[600]	valid_0's l2: 0.351155
[601]	valid_0's l2: 0.351131
[602]	valid_0's l2: 0.351131
[603]	valid_0's l2: 0.351103
[604]	valid_0's l2: 0.351097
[605]	valid_0's l2: 0.351073
[606]	valid_0's l2: 0.351051
[607]	valid_0's l2: 0.351028
[608]	valid_0's l2: 0.351026
[609]	valid_0's l2: 0.351023
[610]	valid_0's l

[861]	valid_0's l2: 0.346313
[862]	valid_0's l2: 0.346316
[863]	valid_0's l2: 0.346314
[864]	valid_0's l2: 0.34631
[865]	valid_0's l2: 0.346285
[866]	valid_0's l2: 0.346282
[867]	valid_0's l2: 0.346243
[868]	valid_0's l2: 0.346226
[869]	valid_0's l2: 0.346224
[870]	valid_0's l2: 0.346221
[871]	valid_0's l2: 0.346218
[872]	valid_0's l2: 0.346217
[873]	valid_0's l2: 0.346215
[874]	valid_0's l2: 0.346203
[875]	valid_0's l2: 0.346204
[876]	valid_0's l2: 0.346183
[877]	valid_0's l2: 0.346157
[878]	valid_0's l2: 0.346153
[879]	valid_0's l2: 0.346133
[880]	valid_0's l2: 0.346097
[881]	valid_0's l2: 0.346073
[882]	valid_0's l2: 0.346048
[883]	valid_0's l2: 0.346025
[884]	valid_0's l2: 0.346005
[885]	valid_0's l2: 0.345979
[886]	valid_0's l2: 0.345948
[887]	valid_0's l2: 0.345929
[888]	valid_0's l2: 0.345928
[889]	valid_0's l2: 0.345926
[890]	valid_0's l2: 0.345874
[891]	valid_0's l2: 0.345875
[892]	valid_0's l2: 0.345871
[893]	valid_0's l2: 0.345867
[894]	valid_0's l2: 0.345864
[895]	valid_0's

[1142]	valid_0's l2: 0.342589
[1143]	valid_0's l2: 0.342564
[1144]	valid_0's l2: 0.342552
[1145]	valid_0's l2: 0.342528
[1146]	valid_0's l2: 0.342507
[1147]	valid_0's l2: 0.342501
[1148]	valid_0's l2: 0.342485
[1149]	valid_0's l2: 0.342483
[1150]	valid_0's l2: 0.342475
[1151]	valid_0's l2: 0.342462
[1152]	valid_0's l2: 0.342451
[1153]	valid_0's l2: 0.34245
[1154]	valid_0's l2: 0.342443
[1155]	valid_0's l2: 0.342424
[1156]	valid_0's l2: 0.342403
[1157]	valid_0's l2: 0.342404
[1158]	valid_0's l2: 0.342402
[1159]	valid_0's l2: 0.342383
[1160]	valid_0's l2: 0.342364
[1161]	valid_0's l2: 0.342364
[1162]	valid_0's l2: 0.34236
[1163]	valid_0's l2: 0.342353
[1164]	valid_0's l2: 0.342347
[1165]	valid_0's l2: 0.342344
[1166]	valid_0's l2: 0.34234
[1167]	valid_0's l2: 0.34233
[1168]	valid_0's l2: 0.342326
[1169]	valid_0's l2: 0.342314
[1170]	valid_0's l2: 0.34228
[1171]	valid_0's l2: 0.342267
[1172]	valid_0's l2: 0.342265
[1173]	valid_0's l2: 0.342249
[1174]	valid_0's l2: 0.342249
[1175]	valid_0'

[1419]	valid_0's l2: 0.339603
[1420]	valid_0's l2: 0.339598
[1421]	valid_0's l2: 0.339586
[1422]	valid_0's l2: 0.339585
[1423]	valid_0's l2: 0.339581
[1424]	valid_0's l2: 0.339567
[1425]	valid_0's l2: 0.339552
[1426]	valid_0's l2: 0.339548
[1427]	valid_0's l2: 0.339537
[1428]	valid_0's l2: 0.339506
[1429]	valid_0's l2: 0.339505
[1430]	valid_0's l2: 0.339502
[1431]	valid_0's l2: 0.339492
[1432]	valid_0's l2: 0.339483
[1433]	valid_0's l2: 0.339472
[1434]	valid_0's l2: 0.339467
[1435]	valid_0's l2: 0.339468
[1436]	valid_0's l2: 0.339461
[1437]	valid_0's l2: 0.339418
[1438]	valid_0's l2: 0.339408
[1439]	valid_0's l2: 0.339409
[1440]	valid_0's l2: 0.339405
[1441]	valid_0's l2: 0.339405
[1442]	valid_0's l2: 0.339404
[1443]	valid_0's l2: 0.339396
[1444]	valid_0's l2: 0.339374
[1445]	valid_0's l2: 0.339368
[1446]	valid_0's l2: 0.339364
[1447]	valid_0's l2: 0.339357
[1448]	valid_0's l2: 0.339337
[1449]	valid_0's l2: 0.339329
[1450]	valid_0's l2: 0.339325
[1451]	valid_0's l2: 0.339323
[1452]	val

[1694]	valid_0's l2: 0.337502
[1695]	valid_0's l2: 0.337501
[1696]	valid_0's l2: 0.337492
[1697]	valid_0's l2: 0.337488
[1698]	valid_0's l2: 0.337482
[1699]	valid_0's l2: 0.337482
[1700]	valid_0's l2: 0.337477
[1701]	valid_0's l2: 0.337474
[1702]	valid_0's l2: 0.337467
[1703]	valid_0's l2: 0.337456
[1704]	valid_0's l2: 0.337421
[1705]	valid_0's l2: 0.337421
[1706]	valid_0's l2: 0.337415
[1707]	valid_0's l2: 0.337415
[1708]	valid_0's l2: 0.337413
[1709]	valid_0's l2: 0.337411
[1710]	valid_0's l2: 0.337409
[1711]	valid_0's l2: 0.337402
[1712]	valid_0's l2: 0.337402
[1713]	valid_0's l2: 0.337402
[1714]	valid_0's l2: 0.337393
[1715]	valid_0's l2: 0.337383
[1716]	valid_0's l2: 0.33738
[1717]	valid_0's l2: 0.337378
[1718]	valid_0's l2: 0.337378
[1719]	valid_0's l2: 0.337374
[1720]	valid_0's l2: 0.337364
[1721]	valid_0's l2: 0.337344
[1722]	valid_0's l2: 0.337341
[1723]	valid_0's l2: 0.337315
[1724]	valid_0's l2: 0.337311
[1725]	valid_0's l2: 0.337305
[1726]	valid_0's l2: 0.337285
[1727]	vali

In [41]:
y_pred = gbm.predict(X_valid, num_iteration=gbm.best_iteration)
#y_pred = np.argmax(y_pred, axis=1)

In [42]:
y_pred

array([ 2.31565148,  2.47586269,  2.37553958, ...,  1.19048797,
        1.33693581,  1.00767751])

# predict

In [51]:
test = pd.read_csv("~/environment/test.tsv", sep="\t")
events = pd.read_csv("~/environment/events.tsv", sep="\t")

In [52]:
print(users.shape)
users.head()

(69667, 5)


,user_id,age,gender,prefecture,created_on
0,1,34,女性,静岡県,2012-06-26
1,2,31,男性,鳥取県,2012-07-12
2,3,32,男性,東京都,2012-07-15
3,4,30,男性,東京都,2012-07-17
4,5,29,女性,埼玉県,2012-07-17


In [53]:
user_list = test["user_id"]
event_list = events["event_id"]

In [58]:
events.loc[:,"user_id"] = 1
user_data = pd.merge(events, users, on = "user_id")
data = data.rename(columns = {"prefecture_x": "event_prefercture", "prefecture_y":"user_prefecture"})

data.loc[:,"event_date"] = pd.to_datetime(data["event_start_at"])
data.loc[:,"event_year"] = data.loc[:,"event_date"].apply(lambda x:str(x.year))
data.loc[:,"event_month"] = data.loc[:,"event_date"].apply(lambda x:str(x.month))
data.loc[:,"event_day"] = data.loc[:,"event_date"].apply(lambda x:str(x.day))
data.loc[:,"event_weekday"] = data.loc[:,"event_date"].apply(lambda x:str(x.dayofweek))
data.loc[:,"event_hour"] = data.loc[:,"event_date"].apply(lambda x:str(x.hour))

data.loc[:,"user_date"] = pd.to_datetime(data["created_on"])
data.loc[:,"user_year"] = data.loc[:,"user_date"].apply(lambda x:str(x.year))
data.loc[:,"user_month"] = data.loc[:,"user_date"].apply(lambda x:str(x.month))

data.head()

,user_id,event_id,time_stamp,action_type,num_of_people,payment_method,total_price,user_id_str,event_id_str,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_start_at,event_prefercture,first_published_at,female_price,male_price,interest,age,gender,user_prefecture,created_on,event_date,event_year,event_month,event_day,event_weekday,event_hour,user_date,user_year,user_month
0,1,6261,2015-03-19 09:15:50,3,1.0,クレカ,4000.0,1,6261,27,39.0,27,39.0,2015-03-22 14:00:00,東京都,NaN,4000.0,6500.0,NaN,34,女性,静岡県,2012-06-26,2015-03-22 14:00:00,2015,3,22,6,14,2012-06-26,2012,6
1,1,127600,2017-06-05 12:33:17,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
2,1,127600,2017-06-05 12:37:09,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
3,1,127600,2017-06-05 12:32:04,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6
4,1,127600,2017-06-06 09:37:45,1,NaN,NaN,NaN,1,127600,24,36.0,26,38.0,2017-06-11 10:00:00,埼玉県,2017-05-01 14:57:33,2000.0,6000.0,NaN,34,女性,静岡県,2012-06-26,2017-06-11 10:00:00,2017,6,11,6,10,2012-06-26,2012,6


In [32]:
X = data[features]
print(X.shape)
X.head()

(1442610, 17)


,female_age_lower,female_age_upper,male_age_lower,male_age_upper,event_prefecture,female_price,male_price,age,gender,user_prefecture,event_year,event_month,event_day,event_weekday,event_hour,user_year,user_month
0,27,39.0,27,39.0,東京都,4000.0,6500.0,34,女性,静岡県,2015,3,22,6,14,2012,6
1,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
2,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
3,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6
4,24,36.0,26,38.0,埼玉県,2000.0,6000.0,34,女性,静岡県,2017,6,11,6,10,2012,6


In [33]:
e_pre = prefecture_labeling(X["event_prefecture"])
e_pre.columns = "event_" + e_pre.columns.values
u_pre = prefecture_labeling(X["user_prefecture"])
u_pre.columns = "user_" + u_pre.columns.values
gen = gender_labeling(X["gender"])

temp = X[list(set(features) - set(["user_prefecture", "event_prefecture", "gender"]))]
print(temp.shape)
temp = pd.concat((temp,gen), axis = 1)
temp = pd.concat((temp,e_pre), axis = 1)
temp = pd.concat((temp,u_pre), axis = 1)

X = temp
print(X.shape)
X.head()

(1442610, 14)
(1442610, 17)


,male_age_lower,user_month,user_year,male_price,female_age_upper,female_age_lower,event_year,female_price,event_month,age,male_age_upper,event_day,event_weekday,event_hour,gender,event_prefecture,user_prefecture
0,27,6,2012,6500.0,39.0,27,2015,4000.0,3,34,39.0,22,6,14,0,27,43
1,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
2,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
3,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
4,26,6,2012,6000.0,36.0,24,2017,2000.0,6,34,38.0,11,6,10,0,8,43
